# 5 Multi-Image Dictionary

## 5.1 Imports & Constants

In [1]:
! pip install opencv-python

In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import cv2

from library import generator
from cifar10_web import cifar10

%config InlineBackend.figure_format='retina'

## 5.2 Utilities

In [3]:
def patch_to_image(pixels, patch_size, channels):
    channel_patches = np.split(pixels, channels)
    for channel in range(channels):
        channel_patches[channel] = np.reshape(channel_patches[channel], (patch_size, patch_size))
    patch = np.dstack(channel_patches)
    return patch

## 5.3 Figure Components

In [4]:
import os
firstFile=os.listdir("Untitled Folder/")
imageFile=cv2.imread("Untitled Folder/"+firstFile[0],0)
img=np.zeros((imageFile.shape[0],imageFile.shape[1],573))
i=0
for file_name in os.listdir("Untitled Folder/"):
    if file_name.split(".")[-1].lower() in {"jpeg", "jpg", "png"}:
        img[:,:,i] = cv2.imread("Untitled Folder/" + file_name,0)
        i=i+1
        
        
#Crop the img so that the sides are divisible by 8 for 8x8 patches
#imgCrop=img[0:8*int(imageFile.shape[1]/8),0:8*int(imageFile.shape[1]/8),:]
#split into quadrants
imageSize=8*int(imageFile.shape[1]/8)
imgCrop=img[:int(imageSize*0.25),0:int(imageSize*0.25),:]

In [5]:
CIFAR_SIZE = 36 #size of the image
CIFAR_CHANNELS = 1 #changed from 3 

#train_images, train_labels, _, _ = cifar10(path=None)
train_images=imgCrop
total_images = 573 #len(train_images)
print(imgCrop.shape)

(36, 36, 573)


In [6]:
CHANNELS = 1
PATCH_SIZE = 32
TOTAL_PATCHES = 10000

observations = np.zeros((PATCH_SIZE * PATCH_SIZE * CHANNELS, TOTAL_PATCHES))
jj=0
for col in range(TOTAL_PATCHES):
    image_index = np.random.randint(total_images)
    image = patch_to_image(train_images[:,:,image_index], CIFAR_SIZE, CIFAR_CHANNELS)
    x_start = np.random.randint(CIFAR_SIZE - PATCH_SIZE)
    y_start = np.random.randint(CIFAR_SIZE - PATCH_SIZE)
    sample_patch = image[x_start:x_start + PATCH_SIZE, y_start:y_start + PATCH_SIZE,:]
    #sample_patch=train_images[:,:,jj]
    
    slices = []
    for channel in range(CHANNELS):
        current = sample_patch[:,:]
        slices.append(np.ndarray.flatten(current))
    observations[:,col] = np.concatenate(slices, axis=None)
    #jj=jj+1

In [7]:
ITERATIONS = 200
#observations=np.reshape(imgCrop[:,:,10].flatten,(360*360,10))
updates = generator.get_dictionary_learning_iterates(observations)
dictionary = next(itertools.islice(updates, ITERATIONS, None))
dictionary = dictionary.T
encoding = dictionary.T @ observations

In [8]:
### CIFAR BASES

#Note only include this if using full
#PATCH_SIZE=360

norms = np.abs(encoding)
norms = np.sum(norms, axis=1)
all_indices = list(range(len(norms)))
all_indices.sort(key=lambda num: norms[num], reverse=True)

sum_signs = np.sum(encoding, axis=1)
sum_signs = np.sign(sum_signs)

ROWS, COLS = 12, 16

fig, axs = plt.subplots(ROWS, COLS, figsize=(64, 48))
plt.subplots_adjust(left=None, right=None, bottom=None, top=None, wspace=0.05, hspace=0.05)
for index, ax in zip(all_indices[: ROWS * COLS], axs.flat):
    base = dictionary[:,index] * sum_signs[index]
    base = base - base.min()
    base = base / base.max()
    base = patch_to_image(base, PATCH_SIZE, CHANNELS)
    
    ax.imshow(base)
    ax.axis('off')
fig.savefig('05-Dictionary_Difference_images_upper_left.pdf', bbox_inches='tight')
plt.close()